In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [7]:
# Hyperparameters

input_size = 28 #28x28 images
sequence_length = 28
num_layers = 2
hidden_size = 256 #nodes
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [16]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # Size = (batch_size, sequencexfeatures)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) # hidden state
        
        # Forward prop
        
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out
        
        

In [17]:
train_dataset = datasets.MNIST('./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST('./data', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [18]:
model = RNN(input_size, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        
        data = data.squeeze(1)
        
        #forward pass
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

In [21]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking training accuracy")
        
        t = "Train"
        
    else:
        print("checking test accuracy")
        
        t = "Test"
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.squeeze(1)
            
            scores = model(x)
            
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f"{t} accuracy: {float(num_correct)/ float(num_samples)*100:.2f}")
        
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking training accuracy
Train accuracy: 97.27
checking test accuracy
Test accuracy: 97.23
